In [2]:
import threading
import time
import gensim.downloader as api
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import os
from gensim.models.keyedvectors import KeyedVectors
from PIL import Image, ImageTk
import re
import jellyfish
from typing import List, Set
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModelForSequenceClassification, AdamW
import torch
from bs4 import BeautifulSoup
from fuzzywuzzy import fuzz

In [3]:
stop_event = threading.Event()
# Kiểu mô hình mặc định
default_model = "PhoBERT"
model = None
tokenizer = None
word2vec_model = None
doc2vec_model = None
test1_path = None
test2_path = None
# độ dài mã thông báo
MAX_LENGTH = 512
local_model_dir = "./models"
matrix_file_path1 = None
matrix_file_path2 = None

In [4]:
file_path = 'vietnamese-stopwords.txt'
with open(file_path, 'r', encoding='utf-8') as file:
    STOP_WORDS = set(file.read().splitlines())

print(STOP_WORDS)

{'bỗng chốc', 'dùng hết', 'buổi mới', 'vài', 'cho về', 'xăm xúi', 'chầm chập', 'bị chú', 'trước kia', 'tắp tắp', 'vâng vâng', 'với', 'chăng', 'nghe nói', 'hoặc', 'lại', 'lâu nay', 'để cho', 'vừa khi', 'phỏng', 'cho được', 'nghe lại', 'hỗ trợ', 'cô quả', 'áng như', 'bằng người', 'bất quá chỉ', 'so', 'để giống', 'nhìn xuống', 'cả nghĩ', 'thay đổi', 'cần', 'gần ngày', 'chứ còn', 'trong', 'bấy chầy', 'do vì', 'vô hình trung', 'nước xuống', 'đặt ra', 'dưới', 'tính người', 'đưa chuyện', 'nhìn nhận', 'nhờ', 'bức', 'nữa', 'thấp cơ', 'lại quả', 'gần như', 'cùng ăn', 'sao cho', 'cơ chừng', 'trả trước', 'phía trong', 'như sau', 'nhìn lại', 'đâu cũng', 'chỉ chính', 'dùng', 'mọi khi', 'thoạt nghe', 'nhỏ người', 'ai nấy', 'đã thế', 'lấy lý do', 'ngoài xa', 'năm', 'ô kê', 'có điều kiện', 'mới hay', 'amen', 'tiếp theo', 'như thế', 'thanh thanh', 'nào là', 'thế', 'nghe được', 'tuổi tôi', 'đều bước', 'sẽ biết', 'để được', 'quá giờ', 'vì rằng', 'nhà ngươi', 'hơn hết', 'dù', 'phỏng như', 'bỏ', 'chứ không 

In [5]:
def remove_stopwords(text, stopwords):
    words = text.split()
    filtered_words = [word for word in words if word not in stopwords]
    return ' '.join(filtered_words)

In [6]:
df = pd.read_csv('D:/Python/Quora/Quora/quora-question-pairs/Data/Dataset/train/tiennvo1.csv')

In [7]:
new_df = df

In [8]:
new_df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,Tháng 8 còn trường nào xét học bạ không?,Những trường Đại Học còn nhận học bạ trong thá...,1
1,1,3,4,Chính sách hỗ trợ học phí cho sinh viên sư phạ...,Những điều cần biết về chính sách hỗ trợ học p...,1
2,2,5,6,Trường ĐHSP có cho thiếu học bạ trong hồ sơ nh...,Có thể bổ sung học bạ sau khi nộp hồ sơ vào ĐH...,1
3,3,7,8,Chương trình học ngành SP Nga có phải 100% tiế...,Xuyên suốt bài giảng ngành SP Nga có phải 100%...,1
4,4,9,10,Không thuộc đối tượng ưu tiên có thể đăng kí ở...,Điều kiện đăng ký ở ký túc xá của trường?,1


In [9]:
def preprocess(q):
    
    q = str(q).lower().strip()
    
    # Replace certain special characters with their string equivalents
    q = q.replace('%', ' percent')
    q = q.replace('$', ' dollar ')
    q = q.replace('₹', ' rupee ')
    q = q.replace('€', ' euro ')
    q = q.replace('@', ' at ')
    
    # The pattern '[math]' appears around 900 times in the whole dataset.
    q = q.replace('[math]', '')
    
    # Replacing some numbers with string equivalents (not perfect, can be done better to account for more cases)
    q = q.replace(',000,000,000 ', 'b ')
    q = q.replace(',000,000 ', 'm ')
    q = q.replace(',000 ', 'k ')
    q = re.sub(r'([0-9]+)000000000', r'\1b', q)
    q = re.sub(r'([0-9]+)000000', r'\1m', q)
    q = re.sub(r'([0-9]+)000', r'\1k', q)
    
    contractions = { 
    "sp": "sư phạm",
    "sn": "sinh năm",
    "bn": "bao nhiêu",
    "ngta": "người ta",
    "lm": "làm",
    "khg": "không",
    "ko": "không",
    "hok": "không",
    "khum": "không",
    "kg": "không",
    "đc": "được",
    "dc": "được",
    "đg": "đang",
    "ng": "người",
    "bt": "biết",
    "h": "giờ",
    "hc": "học",
    "vt": "viết",
    "vc": "việc",
    "trc": "trước",
    "j": "gì",
    "xg": "xong",
    "bx": "bữa",
    "vg": "vâng",
    "v": "vậy",
    "m": "mày",
    "t": "tôi",
    "s": "sao",
    "r": "rồi",
    "chs": "chơi",
    "ae": "anh em",
    "a": "anh",
    "e": "em",
    "cj": "chị",
    "đhsp": "đại học sư phạm",
    "đh": "đại học",
    "đhspkt": "đại học sư phạm kỹ thuật",
    "ktx": "kí túc xá",
    "đt": "điện thoại",
    "cntt": "công nghệ thông tin",
    "tp": "thành phố",
    "hcm": "hồ chí minh",
    "ain't": "am not",
    "aren't": "are not",
    "can't": "can not",
    "can't've": "can not have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have"
    }

    q_decontracted = []

    for word in q.split():
        if word in contractions:
            word = contractions[word]

        q_decontracted.append(word)

    q = ' '.join(q_decontracted)
    q = q.replace("'ve", " have")
    q = q.replace("n't", " not")
    q = q.replace("'re", " are")
    q = q.replace("'ll", " will")
    
    # Removing HTML tags
    q = BeautifulSoup(q)
    q = q.get_text()
    # Remove punctuations
    pattern = re.compile('\W')
    q = re.sub(pattern, ' ', q).strip()

    
    return q

In [10]:
new_df['question1'] = new_df['question1'].apply(preprocess)
new_df['question2'] = new_df['question2'].apply(preprocess)

c:\Users\Tienn Vo\AppData\Local\Programs\Python\Python311\Lib\site-packages\bs4\__init__.py:332: MarkupResemblesLocatorWarning: "." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [11]:
new_df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,tháng 8 còn trường nào xét học bạ không,những trường đại học còn nhận học bạ trong thá...,1
1,1,3,4,chính sách hỗ trợ học phí cho sinh viên sư phạ...,những điều cần biết về chính sách hỗ trợ học p...,1
2,2,5,6,trường đại học sư phạm có cho thiếu học bạ tro...,có thể bổ sung học bạ sau khi nộp hồ sơ vào đạ...,1
3,3,7,8,chương trình học ngành sư phạm nga có phải 100...,xuyên suốt bài giảng ngành sư phạm nga có phải...,1
4,4,9,10,không thuộc đối tượng ưu tiên có thể đăng kí ở...,điều kiện đăng ký ở ký túc xá của trường,1


In [12]:
# def fetch_fuzzy_features(row):
    
#     q1 = row['question1']
#     q2 = row['question2']
    
#     fuzzy_features = [0.0]*4
    
#     # fuzz_ratio
#     fuzzy_features[0] = fuzz.QRatio(q1, q2)

#     # fuzz_partial_ratio
#     fuzzy_features[1] = fuzz.partial_ratio(q1, q2)

#     # token_sort_ratio
#     fuzzy_features[2] = fuzz.token_sort_ratio(q1, q2)

#     # token_set_ratio
#     fuzzy_features[3] = fuzz.token_set_ratio(q1, q2)

#     return fuzzy_features

In [13]:
# fuzzy_features = new_df.apply(fetch_fuzzy_features, axis=1)

# # Creating new feature columns for fuzzy features
# new_df['fuzz_ratio'] = list(map(lambda x: x[0], fuzzy_features))
# new_df['fuzz_partial_ratio'] = list(map(lambda x: x[1], fuzzy_features))
# new_df['token_sort_ratio'] = list(map(lambda x: x[2], fuzzy_features))
# new_df['token_set_ratio'] = list(map(lambda x: x[3], fuzzy_features))

In [14]:
def ensure_dir_exists(dir_path):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

In [15]:
ensure_dir_exists(local_model_dir)

In [16]:
def load_or_download_phobert():
    phobert_model_path = os.path.join(local_model_dir, "models--vinai--phobert-base")
    if not os.path.exists(phobert_model_path):
        print("Downloading PhoBERT model...")
        AutoModel.from_pretrained("vinai/phobert-base", cache_dir=local_model_dir)
        AutoTokenizer.from_pretrained("vinai/phobert-base", cache_dir=local_model_dir)
    else:
        print("PhoBERT model loaded from local storage.")
    return AutoModel.from_pretrained(
        "vinai/phobert-base", cache_dir=local_model_dir
    ), AutoTokenizer.from_pretrained("vinai/phobert-base", cache_dir=local_model_dir)

In [17]:
def preload_models():
    global model, tokenizer
    try:
        if not os.path.exists(local_model_dir):
            os.makedirs(local_model_dir)
            print(f"Directory created: {local_model_dir}")
        ensure_dir_exists(local_model_dir)
        model, tokenizer = load_or_download_phobert()
        print(f"Models loaded successfully.")
        time.sleep(1)
    except Exception as e:
        print(f"Error loading models: {str(e)}")

In [18]:
#Chuẩn hóa ma trận tương đồng về khoảng [0, 1].
def normalize_similarity_matrix(matrix):
    return (matrix + 1) / 2

In [19]:
#Trích xuất embedding của câu bằng PhoBERT.
def get_embedding(sentence):
    inputs = tokenizer(
        sentence, 
        return_tensors="pt", 
        truncation=True, 
        padding=True, 
        max_length=128
    )
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().detach().numpy()

In [20]:
#Tính độ tương đồng cosine giữa hai câu.
def sentence_similarity(sent1, sent2):
    embed1 = get_embedding(sent1)
    embed2 = get_embedding(sent2)
    cosine_similarity = np.dot(embed1, embed2) / (
        np.linalg.norm(embed1) * np.linalg.norm(embed2)
    )
    return cosine_similarity

In [21]:
#Tính độ tương đồng sử dụng PhoBERT.
def calculate_similarity_phobert(questions):
    try:
        normalized_similarity_matrix = sentence_similarity(questions[0], questions[1])
        # if normalized_similarity_matrix >1:
        #     normalized_similarity_matrix = 1.0
        return normalized_similarity_matrix
    except Exception as e:

        raise RuntimeError(f"Error calculate similarity Phobert: {str(e)}")

In [22]:
#Gọi hàm tính toán độ tương đồng tùy thuộc vào loại mô hình.
def calculate_similarity(questions, model_type):
    questions1 = questions.copy()
    if model_type == "PhoBERT":
        return calculate_similarity_phobert(questions1)

In [23]:
# Khai báo biến toàn cục
data_file_path1 = ""
data_file_path2 = ""
keywords_file_path = ""
matrix_tab3 = []

In [24]:
# Hàm đọc từ khóa từ file Excel
def load_keywords_from_excel(
    file_path: str, sheet_name: str, column_name: str
) -> Set[str]:
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    keywords = set(df[column_name].dropna().str.strip().str.lower())
    return keywords

In [25]:
# Hàm kiểm tra từ khóa dựa trên ngưỡng tương đồng Jaro-Winkler
def is_keyword_present_with_threshold(
    keyword: str, words: List[str], threshold: float = 0.6
) -> int:
    for word in words:
        similarity = jellyfish.jaro_winkler_similarity(keyword, word)
        if similarity >= threshold:
            return 1
    return 0

In [26]:
# Hàm tạo véc-tơ đặc trưng cho câu hỏi
def create_feature_vector(
    words: List[str], keywords: Set[str], threshold: float = 0.6
) -> List[int]:
    feature_vector = [
        is_keyword_present_with_threshold(keyword, words, threshold)
        for keyword in keywords
    ]
    return feature_vector

In [27]:
# Hàm tính độ tương đồng Jaccard
def jaccard_similarity(vec1: List[int], vec2: List[int]) -> float:
    intersection = sum(1 for v1, v2 in zip(vec1, vec2) if v1 == 1 and v2 == 1)
    union = sum(1 for v1, v2 in zip(vec1, vec2) if v1 == 1 or v2 == 1)
    return intersection / union if union != 0 else 0

In [28]:
preload_models()

PhoBERT model loaded from local storage.
Models loaded successfully.


In [29]:
train_df, test_df = train_test_split(new_df, test_size=0.2, random_state=42)

In [30]:
def preprocess_data(new_df):
    inputs = tokenizer(
        new_df['question1'].tolist(),
        new_df['question2'].tolist(),
        truncation=True,
        padding=True,
        max_length=128,
        return_tensors="pt"
    )
    inputs['labels'] = new_df['is_duplicate'].tolist()
    return inputs

In [31]:
def find_answer(input_question, threshold=0.8):
    # Tính độ tương đồng giữa câu hỏi đầu vào và các câu hỏi trong bộ dữ liệu
    max_similarity = -1
    best_match = None

    for index, row in df.iterrows():
        # Tính độ tương đồng giữa input_question và question1
        similarity1 = calculate_similarity([input_question, row['question1']], "PhoBERT")
        # Tính độ tương đồng giữa input_question và question2
        similarity2 = calculate_similarity([input_question, row['question2']], "PhoBERT")
        # Lấy độ tương đồng cao nhất
        max_current_similarity = max(similarity1, similarity2)

        # Nếu độ tương đồng cao hơn ngưỡng và cao hơn giá trị hiện tại
        if max_current_similarity > threshold and max_current_similarity > max_similarity:
            max_similarity = max_current_similarity
            best_match = row

    # Nếu tìm thấy câu hỏi trùng khớp, trả về câu trả lời
    if best_match is not None:
        return best_match['answer']
    else:
        return "Không tìm thấy câu trả lời phù hợp."

In [32]:
# Áp dụng tiền xử lý
train_data = preprocess_data(train_df)
test_data = preprocess_data(test_df)

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [33]:
class QuestionPairDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

In [34]:
# Tạo Dataset
train_dataset = QuestionPairDataset(train_data, train_df['is_duplicate'].tolist())
test_dataset = QuestionPairDataset(test_data, test_df['is_duplicate'].tolist())

In [35]:
# Tạo DataLoader
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [36]:
# Load mô hình PhoBERT với đầu ra là phân loại nhị phân
model = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base", num_labels=2)

# Thiết lập optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Tienn Vo\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [37]:
if torch.cuda.is_available():
    print("GPU is available and being used.")
else:
    print("GPU is not available, using CPU instead.")

GPU is available and being used.


In [38]:
# Huấn luyện mô hình
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(5):  # Số epoch
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_loader)}")

Epoch 1, Loss: 0.4707938263444546
Epoch 2, Loss: 0.35735008051328976
Epoch 3, Loss: 0.2709968293134617
Epoch 4, Loss: 0.1885770927961918
Epoch 5, Loss: 0.13244741231448887


In [39]:
#đánh giá mô hình
from sklearn.metrics import accuracy_score

model.eval()
predictions, true_labels = [], []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1).cpu().numpy()

        predictions.extend(preds)
        true_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(true_labels, predictions)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.8261


In [40]:
model.save_pretrained("./phobert-finetuned")
tokenizer.save_pretrained("./phobert-finetuned")

('./phobert-finetuned\\tokenizer_config.json',
 './phobert-finetuned\\special_tokens_map.json',
 './phobert-finetuned\\vocab.txt',
 './phobert-finetuned\\bpe.codes',
 './phobert-finetuned\\added_tokens.json')

In [41]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Tải lại mô hình và tokenizer
model = AutoModelForSequenceClassification.from_pretrained("./phobert-finetuned")
tokenizer = AutoTokenizer.from_pretrained("./phobert-finetuned")

# Dự đoán độ tương đồng
def predict_similarity(question1, question2):
    inputs = tokenizer(question1, question2, return_tensors="pt", truncation=True, padding=True, max_length=128)
    outputs = model(**inputs)
    probs = torch.softmax(outputs.logits, dim=1)
    return probs[0][1].item()  # Xác suất thuộc lớp tương đồng (1)

In [42]:
#preload_models()

In [43]:
# Ví dụ
question1 = "Trang Facebook nào nhất ở Ấn Độ?"
question2 = "Khi nào Facebook ra mắt ở Ấn Độ?"

In [44]:
# try:
#     similarity_score = calculate_similarity([question1, question2], "PhoBERT")
#     print(f"Độ tương đồng giữa hai câu hỏi là: {similarity_score:.4f}")
# except Exception as e:
#     print(f"Lỗi khi tính toán độ tương đồng: {str(e)}")

In [54]:
test_df = new_df.sample(10)
test_df['similarity_score'] = test_df.apply(lambda row: predict_similarity(row['question1'], row['question2']), axis=1)
test_df

,id,qid1,qid2,question1,question2,is_duplicate,similarity_score
22107,315191,126408,440050,tại sao tôi không còn thông báo về nhận xét về...,tại sao tôi không còn được thông báo với một t...,1,0.788766
19757,366881,329721,67805,có phải cạo râu là một cách tốt để có thêm tóc...,làm thế nào để một người mọc tóc trên khuôn mặt,1,0.941409
6836,162134,252467,78437,có khó để học tiếng anh không,làm thế nào khó khăn để học tiếng anh,0,0.981169
16643,157611,230470,246398,tại sao động đất xảy ra,động đất chủ yếu xảy ra ở đâu,0,0.110391
8397,380838,512504,512505,tôi đang học đại học và tôi không có tiền làm ...,tôi thú nhận cảm xúc của mình với một anh chàn...,0,0.000720
11795,324525,450634,450635,một số ví dụ về công nghệ cạnh hàng đầu là gì,một số ví dụ về các công nghệ cạnh hàng đầu là gì,1,0.997555
757,757,1515,1516,sinh viên có thể xin chuyển từ hệ đại học sang...,trường có cho phép sinh viên chuyển từ chương ...,1,0.998178
5246,243799,356298,356299,nếu người ta phải lựa chọn giữa cse trong mani...,tôi đang chọn manipal cse thay vì vit cse đó c...,0,0.000959
25045,251487,365606,365607,làm cách nào để tăng bài đăng trên trang fb củ...,làm cách nào để tăng cường đăng trang fb của t...,1,0.998520
14225,20379,38451,38452,tại sao chính phủ ấn độ không nên bãi bỏ thuế ...,một số điểm tham quan ít được biết đến để xem ...,0,0.001466
